## Sarcasm Detection Tuning:

In [ ]:
# mount google drive and setup Kaggle
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

project_path = "/content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/"


%cd project_path

import os
# Download kaggle.json from https://www.kaggle.com/settings/account
# place kaggle.json in the below directory
os.environ['KAGGLE_CONFIG_DIR'] = project_path

Mounted at /content/gdrive
[Errno 2] No such file or directory: 'project_path'
/content


In [ ]:
# download data from Kaggle
import os

file_path = project_path+"Sarcasm_Headlines_Dataset_v2.json"

# Check if the file exists
if os.path.isfile(file_path):
  print(f"✅ Success: The file '{os.path.basename(file_path)}' was found at the specified path.")

else:
  print(f"❌ Error: The file '{os.path.basename(file_path)}' was NOT found.")

  # Download the dataset
  !kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection


  # Unzip the downloaded file
  !unzip news-headlines-dataset-for-sarcasm-detection.zip

  !mkdir -p "$project_path"

  !mv Sarcasm_Headlines_Dataset_v2.json "$project_path"

✅ Success: The file 'Sarcasm_Headlines_Dataset_v2.json' was found at the specified path.


In [ ]:
import json
import re
import random
import numpy as np
import pandas as pd
import torch
import os

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

from nltk.corpus import wordnet
import nltk
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

nltk.download('wordnet')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Data Preparation (Loading, Cleaning, Augmentation, Tokenization)

In [ ]:
# --- Data Loading and Preprocessing ---
#path = "Sarcasm_Headlines_Dataset_v2.json"
path = file_path
data = []

with open(path, "r") as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Skipping malformed JSON line: {line.strip()}. Error: {e}")
df = pd.DataFrame(data)
df = df[["headline", "is_sarcastic"]]

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)  # remove special chars
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_headline"] = df["headline"].apply(clean_text)

def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([w for w in words if len(w) > 3]))

    if len(random_word_list) == 0:
        return sentence

    for _ in range(n):
        # Choose a random word from the list
        word_to_replace = random.choice(random_word_list)
        synonyms = wordnet.synsets(word_to_replace)
        if not synonyms:
            continue

        # Choose a random synonym for better diversity
        synonym_words = random.choice(synonyms).lemma_names()
        synonym = synonym_words[0].replace("_", " ")

        # Find the index of the word to replace in the original words list
        # This handles multiple occurrences, replacing only the first one found.
        try:
            idx = new_words.index(word_to_replace)
            new_words[idx] = synonym
        except ValueError:
            continue # Should not happen if word_to_replace came from words

    return " ".join(new_words)

# Augmentation (only sarcastic data is augmented, as in the original notebook)
augmented_rows = []
for idx, row in df.iterrows():
    if row["is_sarcastic"] == 1:
        aug = synonym_replacement(row["clean_headline"])
        augmented_rows.append({"clean_headline": aug, "is_sarcastic": 1})

aug_df = pd.DataFrame(augmented_rows)
df_augmented = pd.concat([df[["clean_headline", "is_sarcastic"]], aug_df])
df_augmented.reset_index(drop=True, inplace=True)

# Data Splitting
train_df, temp_df = train_test_split(df_augmented, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)




In [ ]:
# --- Setup for Unsloth Llama 3.1 8B Fine-Tuning ---
# Install unsloth and dependencies. Using %%capture to keep the notebook clean.
# NOTE: This cell will take some time to run the first time due to installations.

# Use os.environ check for cleaner install in Colab, similar to the starter notebook
import os, re
from datetime import datetime

# Check for Colab environment to determine installation path
if "COLAB_GPU" in os.environ:
    # Colab-specific installation for faster performance
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    # The xformers version is often tightly coupled to the PyTorch version
    xformers = "xformers==0.0.33.post1" if v=="2.9" else "xformers==0.0.32.post2" if v=="2.8" else "xformers==0.0.29.post3"

    # We use pip directly for the core installs, which can be verbose, so we put it outside a function.
    print("Installing Unsloth dependencies for Colab environment...")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
else:
    # General installation for other environments
    print("Installing Unsloth dependencies...")
    !pip install unsloth

!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments

# Configuration
max_seq_length = 512 # Max context length for the model
dtype = None          # Auto-detect best data type (e.g., bfloat16)
load_in_4bit = True   # Use 4-bit quantization for efficiency

# Load the base model and tokenizer
print("\nLoading unsloth/Meta-Llama-3.1-8B...")
model_llama, tokenizer_llama = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


# The instructional prompt template for training (same as starter notebook)
training_prompt = """You are an expert linguistic analysis agent trained to detect sarcasm. Your task is to analyze the provided text and determine if it is sarcastic or not.

Input Text:
{}.

Output 'True' if the text is sarcastic, otherwise 'False'.

Output:
{}
"""

# Must add an End Of Sequence (EOS) token
EOS_TOKEN = tokenizer_llama.eos_token

# Function to format the dataset to the prompt template
def formatting_prompts_func(examples):
    # Use 'clean_headline' column from the augmented dataset
    headlines = examples["clean_headline"]
    outputs = examples["is_sarcastic"] #

    # Convert numerical labels to "True"/"False" string for Llama instruction-tuning
    outputs = [("True" if x == 1 else "False") for x in outputs]
    texts = []

    for headline, output in zip(headlines, outputs):
        text = training_prompt.format(headline, output) + EOS_TOKEN
        texts.append(text)

    return { "text" : texts }

# Apply the formatting function to the training dataset from BERT_Tuning_Comparison_V2.ipynb
# 'ds' was defined in cell 4.
#df = pd.DataFrame(augmented_rows)
ds = Dataset.from_pandas(df_augmented)

shuffled_dataset = ds.shuffle(seed=42)

# First Split: Separate Train (80%) from the Rest (20%)
train_testvalid = shuffled_dataset.train_test_split(test_size=0.2, seed=42)

# Second Split: Split the 'Rest' (20%) equally into Validation and Test
# Splitting the 20% in half gives you 10% Validation and 10% Test
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

train_dataset = train_testvalid['train'] #.select(range(5000))
validation_dataset = test_valid['train'] #.select(range(1000))
test_dataset = test_valid['test'] #.select(range(1000))

print("Applying formatting to the training dataset...")
formatted_train_dataset_llama = train_dataset.map(formatting_prompts_func, batched=True)


Installing Unsloth dependencies for Colab environment...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/tmp/ipython-input-3082461747.py:29: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.



Loading unsloth/Meta-Llama-3.1-8B...
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Applying formatting to the training dataset...


Map:   0%|          | 0/33802 [00:00<?, ? examples/s]

### Final Comparison of Test Set Metrics

In [ ]:
#Define the Inference and Evaluation Function ---

# The prompt template must match the one used for training EXACTLY up to the 'Output:' line.
EVAL_PROMPT_TEMPLATE = """You are an expert linguistic analysis agent trained to detect sarcasm. Your task is to analyze the provided text and determine if it is sarcastic or not.

Input Text:
{}.

Output 'True' if the text is sarcastic, otherwise 'False'.

Output:
"""

def evaluate_llama_model(model, tokenizer, test_dataset, prompt_template):
    """
    Manually evaluates the Llama model on a test dataset by generating text.
    It compares the generated output ('True' or 'False') with the ground truth label.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Convert 'is_sarcastic' (0 or 1) to 'False' or 'True' string labels
    true_labels_str = [("True" if x == 1 else "False") for x in test_dataset['is_sarcastic']]

    # We will store the extracted predictions here
    predictions_str = []

    print(f"Starting inference on {len(test_dataset)} samples...")

    for i in range(len(test_dataset)):
        headline = test_dataset[i]['clean_headline']

        # Format the prompt for inference
        prompt = prompt_template.format(headline)

        # Tokenize the input prompt
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to(device)

        # Generate a response. We expect a short output ('True' or 'False').
        # max_new_tokens=5 is a safe limit for 'True' or 'False'
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                max_new_tokens=5,
                use_cache=True,
                do_sample=False, # Use deterministic decoding
                pad_token_id=tokenizer.eos_token_id # Important for batching, though we do not batch here
            )

        # Decode the output tokens and slice to get only the new generation
        generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True).strip()

        # Simple extraction of 'True' or 'False' from the generated text
        # In a real-world scenario, you might need more robust parsing.
        predicted_label = 'False'
        if 'True' in generated_text:
             predicted_label = 'True'
        elif 'False' in generated_text:
             predicted_label = 'False'
        # Default to 'False' or handle as an error if neither is found

        predictions_str.append(predicted_label)

        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{len(test_dataset)} samples.")

    # Convert string labels back to numerical (0 or 1) for metric calculation
    # True = 1, False = 0
    true_labels_int = [1 if x == 'True' else 0 for x in true_labels_str]
    predictions_int = [1 if x == 'True' else 0 for x in predictions_str]

    # Calculate metrics
    accuracy = accuracy_score(true_labels_int, predictions_int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels_int,
        predictions_int,
        average='binary',
        pos_label=1 # Calculate for the 'sarcastic' class
    )

    # Placeholder for loss since SFTTrainer doesn't calculate it easily on eval
    eval_loss = None

    return {
        "eval_accuracy": accuracy,
        "eval_precision": precision,
        "eval_recall": recall,
        "eval_f1": f1,
        "eval_loss": eval_loss
    }

test_dataset = test_valid['test']

In [ ]:
from transformers import TrainingArguments
from unsloth import FastLanguageModel
from trl import SFTTrainer
import torch
from datetime import datetime
import pandas as pd
import copy

# Define a single function to encapsulate the entire fine-tuning process
def run_llama_experiment(
    experiment_name,
    train_dataset,
    test_dataset,
    base_model_name,
    lora_r,
    lora_alpha,
    num_train_epochs,
    learning_rate,
    lora_target_modules=None,
    use_rslora=True
):
    print(f"\n======== Starting Experiment: {experiment_name} ========")

    # Load Model and Tokenizer (This may take a moment)
    print("Loading model for experiment...")
    # reload the model fresh for each experiment to ensure isolation.
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = base_model_name,
        max_seq_length = 512, # Consistent max seq length
        dtype = None,
        load_in_4bit = True,
    )

    # Use default target modules if none are specified
    if lora_target_modules is None:
        lora_target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                               "gate_proj", "up_proj", "down_proj"]

    # Setup LoRA adapters
    model = FastLanguageModel.get_peft_model(
        model,
        r = lora_r,
        target_modules = lora_target_modules,
        lora_alpha = lora_alpha,
        lora_dropout = 0, # Keep dropout consistent for now
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 42,
        use_rslora=use_rslora,
    )
    print(f"LoRA Config: r={lora_r}, alpha={lora_alpha}, epochs={num_train_epochs}, lr={learning_rate}")

    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir = f"./{experiment_name.replace(' ', '_').lower()}_"+datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
        num_train_epochs = num_train_epochs,
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        learning_rate = learning_rate,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        report_to = "none",
    )

    # Initialize and Run Trainer
    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = train_dataset,
        dataset_text_field = "text",
        max_seq_length = 512,
        dataset_num_proc = 2,
        packing = False,
        args = training_args,
    )

    # Start training
    trainer.train()

    output_dir = f"{project_path}{experiment_name.replace(' ', '_').lower()}_"+datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    print(f"\nSaving LoRA weights to base model in {output_dir}...")

    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print(f"Model checkpoint and tokenizer saved to: {output_dir}")

    print("\nStarting Test Set Evaluation...")
    # Make a copy of the test dataset just in case the eval modifies it
    test_metrics = evaluate_llama_model(model, tokenizer, test_dataset, EVAL_PROMPT_TEMPLATE)

    # Collate and return results
    results = {
        'Experiment': experiment_name,
        'R_Rank': lora_r,
        'Alpha': lora_alpha,
        'Epochs': num_train_epochs,
        'LR': learning_rate,
        'eval_loss': test_metrics['eval_loss'],
        'eval_accuracy': test_metrics['eval_accuracy'],
        'eval_precision': test_metrics['eval_precision'],
        'eval_recall': test_metrics['eval_recall'],
        'eval_f1': test_metrics['eval_f1']
    }

    return results


In [ ]:
comparison_results_detailed = []

In [ ]:
# base line:
experiment_low_lr = run_llama_experiment(
    experiment_name = 'Llama 3.1 8B base line (LR=2e-4 1 Epoch)',
    train_dataset = formatted_train_dataset_llama,
    test_dataset = test_dataset,
    base_model_name = "unsloth/Meta-Llama-3.1-8B",
    lora_r = 16,
    lora_alpha = 32,
    num_train_epochs = 1,
    learning_rate = 2e-4,
)
comparison_results_detailed.append(experiment_low_lr)


======== Starting Experiment: Llama 3.1 8B base line (LR=2e-4 1 Epoch) ========
Loading model for experiment...
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.12.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


LoRA Config: r=16, alpha=32, epochs=1, lr=0.0002


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/33802 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,802 | Num Epochs = 1 | Total steps = 133
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 4 x 1) = 256
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
50,0.986400
100,0.747900



Saving and Merging LoRA weights to base model in /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_base_line_(lr=2e-4_1_epoch)_2025-12-12_16-38-04...
Model checkpoint and tokenizer saved to: /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_base_line_(lr=2e-4_1_epoch)_2025-12-12_16-38-04

Starting Test Set Evaluation...
Starting inference on 4226 samples...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Processed 100/4226 samples.
Processed 200/4226 samples.
Processed 300/4226 samples.
Processed 400/4226 samples.
Processed 500/4226 samples.
Processed 600/4226 samples.
Processed 700/4226 samples.
Processed 800/4226 samples.
Processed 900/4226 samples.
Processed 1000/4226 samples.
Processed 1100/4226 samples.
Processed 1200/4226 samples.
Processed 1300/4226 samples.
Processed 1400/4226 samples.
Processed 1500/4226 samples.
Processed 1600/4226 samples.
Processed 1700/4226 samples.
Processed 1800/4226 samples.
Processed 1900/4226 samples.
Processed 2000/4226 samples.
Processed 2100/4226 samples.
Processed 2200/4226 samples.
Processed 2300/4226 samples.
Processed 2400/4226 samples.
Processed 2500/4226 samples.
Processed 2600/4226 samples.
Processed 2700/4226 samples.
Processed 2800/4226 samples.
Processed 2900/4226 samples.
Processed 3000/4226 samples.
Processed 3100/4226 samples.
Processed 3200/4226 samples.
Processed 3300/4226 samples.
Processed 3400/4226 samples.
Processed 3500/4226 sam

In [ ]:
# New Experiment 1: Lower LoRA Rank (Test for Efficiency)
experiment_low_r = run_llama_experiment(
    experiment_name = 'Llama 3.1 8B (Low Rank=8)',
    train_dataset = formatted_train_dataset_llama,
    test_dataset = test_dataset,
    base_model_name = "unsloth/Meta-Llama-3.1-8B",
    lora_r = 8,
    lora_alpha = 16,
    num_train_epochs = 1,
    learning_rate = 2e-4,
)
comparison_results_detailed.append(experiment_low_r)



======== Starting Experiment: Llama 3.1 8B (Low Rank=8) ========
Loading model for experiment...
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
LoRA Config: r=8, alpha=16, epochs=1, lr=0.0002


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/33802 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,802 | Num Epochs = 1 | Total steps = 133
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 4 x 1) = 256
 "-____-"     Trainable parameters = 20,971,520 of 8,051,232,768 (0.26% trained)


Step,Training Loss
50,1.057800
100,0.761700



Saving and Merging LoRA weights to base model in /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(low_rank=8)_2025-12-12_17-07-50...
Model checkpoint and tokenizer saved to: /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(low_rank=8)_2025-12-12_17-07-50

Starting Test Set Evaluation...
Starting inference on 4226 samples...
Processed 100/4226 samples.
Processed 200/4226 samples.
Processed 300/4226 samples.
Processed 400/4226 samples.
Processed 500/4226 samples.
Processed 600/4226 samples.
Processed 700/4226 samples.
Processed 800/4226 samples.
Processed 900/4226 samples.
Processed 1000/4226 samples.
Processed 1100/4226 samples.
Processed 1200/4226 samples.
Processed 1300/4226 samples.
Processed 1400/4226 samples.
Processed 1500/4226 samples.
Processed 1600/4226 samples.
Processed 1700/4226 samples.
Processed 1800/4226 samples.
Processed 1900/4226 samples.
Processed 2000/4226 samples.
Processed 2100/4226 samples.
Processed 2200/4226 samples.
Processed 

In [ ]:
# New Experiment 2: Higher Epochs (Test for Overfitting/Convergence)
experiment_high_epochs = run_llama_experiment(
    experiment_name = 'Llama 3.1 8B (2 Epochs)',
    train_dataset = formatted_train_dataset_llama,
    test_dataset = test_dataset,
    base_model_name = "unsloth/Meta-Llama-3.1-8B",
    lora_r = 16,
    lora_alpha = 32,
    num_train_epochs = 2,
    learning_rate = 2e-4,
)
comparison_results_detailed.append(experiment_high_epochs)


======== Starting Experiment: Llama 3.1 8B (2 Epochs) ========
Loading model for experiment...
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
LoRA Config: r=16, alpha=32, epochs=2, lr=0.0002


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/33802 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,802 | Num Epochs = 2 | Total steps = 266
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 4 x 1) = 256
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
50,0.986900
100,0.745600
150,0.688500
200,0.611100
250,0.592100



Saving and Merging LoRA weights to base model in /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(2_epochs)_2025-12-12_17-49-31...
Model checkpoint and tokenizer saved to: /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(2_epochs)_2025-12-12_17-49-31

Starting Test Set Evaluation...
Starting inference on 4226 samples...
Processed 100/4226 samples.
Processed 200/4226 samples.
Processed 300/4226 samples.
Processed 400/4226 samples.
Processed 500/4226 samples.
Processed 600/4226 samples.
Processed 700/4226 samples.
Processed 800/4226 samples.
Processed 900/4226 samples.
Processed 1000/4226 samples.
Processed 1100/4226 samples.
Processed 1200/4226 samples.
Processed 1300/4226 samples.
Processed 1400/4226 samples.
Processed 1500/4226 samples.
Processed 1600/4226 samples.
Processed 1700/4226 samples.
Processed 1800/4226 samples.
Processed 1900/4226 samples.
Processed 2000/4226 samples.
Processed 2100/4226 samples.
Processed 2200/4226 samples.
Processed 2300

In [ ]:
# New Experiment 3: Lower Learning Rate (Test for Generalization)
experiment_low_lr = run_llama_experiment(
    experiment_name = 'Llama 3.1 8B (LR=2e-5)',
    train_dataset = formatted_train_dataset_llama,
    test_dataset = test_dataset,
    base_model_name = "unsloth/Meta-Llama-3.1-8B",
    lora_r = 16,
    lora_alpha = 32,
    num_train_epochs = 1,
    learning_rate = 2e-5,
)
comparison_results_detailed.append(experiment_low_lr)


======== Starting Experiment: Llama 3.1 8B (LR=2e-5) ========
Loading model for experiment...
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
LoRA Config: r=16, alpha=32, epochs=1, lr=2e-05


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/33802 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,802 | Num Epochs = 1 | Total steps = 133
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 4 x 1) = 256
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
50,1.280600
100,0.787600



Saving and Merging LoRA weights to base model in /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(lr=2e-5)_2025-12-12_18-19-46...
Model checkpoint and tokenizer saved to: /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(lr=2e-5)_2025-12-12_18-19-46

Starting Test Set Evaluation...
Starting inference on 4226 samples...
Processed 100/4226 samples.
Processed 200/4226 samples.
Processed 300/4226 samples.
Processed 400/4226 samples.
Processed 500/4226 samples.
Processed 600/4226 samples.
Processed 700/4226 samples.
Processed 800/4226 samples.
Processed 900/4226 samples.
Processed 1000/4226 samples.
Processed 1100/4226 samples.
Processed 1200/4226 samples.
Processed 1300/4226 samples.
Processed 1400/4226 samples.
Processed 1500/4226 samples.
Processed 1600/4226 samples.
Processed 1700/4226 samples.
Processed 1800/4226 samples.
Processed 1900/4226 samples.
Processed 2000/4226 samples.
Processed 2100/4226 samples.
Processed 2200/4226 samples.
Processed 2300/4

In [ ]:
# New Experiment 4:
experiment_low_lr = run_llama_experiment(
    experiment_name = 'Llama 3.1 8B (LR=2e-4 3 Epochs)',
    train_dataset = formatted_train_dataset_llama,
    test_dataset = test_dataset,
    base_model_name = "unsloth/Meta-Llama-3.1-8B",
    lora_r = 32,
    lora_alpha = 64,
    num_train_epochs = 3,
    learning_rate = 2e-4,
)
comparison_results_detailed.append(experiment_low_lr)


======== Starting Experiment: Llama 3.1 8B (LR=2e-4 3 Epochs) ========
Loading model for experiment...
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
LoRA Config: r=32, alpha=64, epochs=3, lr=0.0002


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/33802 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,802 | Num Epochs = 3 | Total steps = 399
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 4 x 1) = 256
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Step,Training Loss
50,0.943000
100,0.739600
150,0.623400
200,0.480600
250,0.453400
300,0.317200
350,0.259900



Saving and Merging LoRA weights to base model in /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(lr=2e-4_3_epochs)_2025-12-12_19-13-21...
Model checkpoint and tokenizer saved to: /content/gdrive/MyDrive/CS-GY 6953 DL/Final Project/llama_3.1_8b_(lr=2e-4_3_epochs)_2025-12-12_19-13-21

Starting Test Set Evaluation...
Starting inference on 4226 samples...
Processed 100/4226 samples.
Processed 200/4226 samples.
Processed 300/4226 samples.
Processed 400/4226 samples.
Processed 500/4226 samples.
Processed 600/4226 samples.
Processed 700/4226 samples.
Processed 800/4226 samples.
Processed 900/4226 samples.
Processed 1000/4226 samples.
Processed 1100/4226 samples.
Processed 1200/4226 samples.
Processed 1300/4226 samples.
Processed 1400/4226 samples.
Processed 1500/4226 samples.
Processed 1600/4226 samples.
Processed 1700/4226 samples.
Processed 1800/4226 samples.
Processed 1900/4226 samples.
Processed 2000/4226 samples.
Processed 2100/4226 samples.
Processed 2200/4226 samples

In [ ]:
# --- Final Display of All Experiments ---
comparison_df_detailed = pd.DataFrame(comparison_results_detailed)
comparison_df_detailed = comparison_df_detailed[['Experiment', 'R_Rank', 'Alpha', 'Epochs', 'LR', 'eval_accuracy', 'eval_precision', 'eval_recall', 'eval_f1']]
comparison_df_detailed.columns = ['Experiment', 'Rank (r)', 'Alpha (α)', 'Epochs', 'LR', 'Accuracy', 'Precision (Sarcastic)', 'Recall (Sarcastic)', 'F1-Score (Sarcastic)']

print("\n\n--- FINAL COMPARISON OF ALL EXPERIMENTS ---")
print(comparison_df_detailed.to_markdown(index=False, floatfmt=(".0f", ".0f", ".0f", ".0f", ".1e", ".4f", ".4f", ".4f", ".4f")))



--- FINAL COMPARISON OF ALL EXPERIMENTS ---
| Experiment                               |   Rank (r) |   Alpha (α) |   Epochs |      LR |   Accuracy |   Precision (Sarcastic) |   Recall (Sarcastic) |   F1-Score (Sarcastic) |
|:-----------------------------------------|-----------:|------------:|---------:|--------:|-----------:|------------------------:|---------------------:|-----------------------:|
| Llama 3.1 8B base line (LR=2e-4 1 Epoch) |         16 |          32 |        1 | 2.0e-04 |     0.9690 |                  0.9680 |               0.9846 |                 0.9762 |
| Llama 3.1 8B (Low Rank=8)                |          8 |          16 |        1 | 2.0e-04 |     0.9655 |                  0.9638 |               0.9835 |                 0.9736 |
| Llama 3.1 8B (2 Epochs)                  |         16 |          32 |        2 | 2.0e-04 |     0.9763 |                  0.9748 |               0.9890 |                 0.9818 |
| Llama 3.1 8B (LR=2e-5)                   |         1

In [ ]:
# Optional. This is to load the previous saved checkpoint to for inference
#
from transformers import AutoTokenizer, AutoModelForCausalLM

# The path where you saved the merged model
output_dir = project_path+"llama_3.1_8b_(low_rank=8)_2025-12-12_14-35-13"

# load the model from saved checkpoint
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = output_dir,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

# Evaluate with test data set
test_metrics = evaluate_llama_model(model, tokenizer, test_dataset, EVAL_PROMPT_TEMPLATE)